In [1]:
# Huggingface token for meta licence acceptance
from dotenv import load_dotenv
import os

load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')
MODEL_ID = "meta-llama/Llama-2-7b-chat-hf"

In [2]:
import lightning as L
from transformers import AutoModelForCausalLM, AutoTokenizer, Conversation
import mlflow
from lightning.pytorch.loggers import MLFlowLogger

/Users/echapman/projects/llm-finetuning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from models.lightning_llama_chat import LitLlamaChat
    
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = LitLlamaChat(
    llm=AutoModelForCausalLM.from_pretrained(MODEL_ID, use_auth_token=HF_TOKEN),
    tokenizer=tokenizer,
    max_new_tokens=2
)

/Users/echapman/projects/llm-finetuning/.venv/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.98s/it]


In [5]:
mlf_logger = MLFlowLogger(
        experiment_name='existing_name', log_model='all'
    )
trainer = L.Trainer(accelerator='cpu', logger=mlf_logger)
input = model.conversations_to_tensors(batch)
mlf_logger.experiment.log_param(
        key='predict_loaded_data_shape',
        value=str(input.shape),
        run_id=mlf_logger.run_id,
    )
trainer.predict(model=model, dataloaders=input)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/echapman/projects/llm-finetuning/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
hug

Predicting DataLoader 0: 100%|██████████| 1/1 [00:31<00:00,  0.03it/s]


[['<s> [INST] <<SYS>>\nAnswer the following questions:\n<</SYS>>\n\nWhat is the capital of England? [/INST]  The']]

In [8]:
import torch
torch.save(model,'model_save.pt')